<a href="https://colab.research.google.com/github/noambassat/SupremeCourtClassifier/blob/main/8_None_relevant_cases_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModel
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import torch
import joblib
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate, cross_val_score, StratifiedKFold,train_test_split,cross_val_predict
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score,make_scorer, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
# from imblearn.under_sampling import RandomUnderSampler

from google.colab import drive


In [ ]:
# התחברות ל-Google Drive
drive.mount('/content/drive')



In [6]:

folder_id = "בית המשפט - קבצים שנבדקו"
directory_path = f"/content/drive/My Drive/{folder_id}"

if not os.path.exists(directory_path):
    print(f"Directory {directory_path} does not exist. Please check the folder path.")
else:
    dataframes = []

    for file_name in os.listdir(directory_path):
        if file_name.endswith(".xlsx") or file_name.endswith(".xls"):
            file_path = os.path.join(directory_path, file_name)
            try:
                df = pd.read_excel(file_path)
                dataframes.append(df)
            except Exception as e:
                print(f"Failed to read {file_name}: {e}")

    if dataframes:
        combined_df = pd.concat(dataframes, ignore_index=True)
        print("All Excel files have been concatenated successfully!")

        output_file = "/content/drive/My Drive/combined_excel.xlsx"  # עדכן את הנתיב לשמירת הקובץ
        combined_df.to_excel(output_file, index=False)
        print(f"Combined file saved at: {output_file}")
    else:
        print("No Excel files found or failed to read.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


All Excel files have been concatenated successfully!
   level_0  index                                          גוף המסמך  \
0        0   2977  ['החלטה בתיק רע"פ 5389/15 בבית המשפט העליון רע...   
1        1   2978  ['החלטה בתיק רע"פ 2487/15 בבית המשפט העליון רע...   
2        2   2979  ['החלטה בתיק רע"פ 4086/15 בבית המשפט העליון רע...   
3        3   2980  ['פסק דין בתיק רע"פ 8866/15 בבית המשפט העליון ...   
4        4   2981  ['החלטה בתיק רע"פ 1094/15 בבית המשפט העליון רע...   

      מספר הליך                                     שם הליך  שנת פתיחת ההליך  \
0  רע"פ 5389/15                                רע"פ 5389/15             2015   
1  רע"פ 2487/15  רע"פ 2487/15 אייל אבולפיה נ. עיריית רמת גנ             2015   
2  רע"פ 4086/15     רע"פ 4086/15 עבד אגבאריה נ. מדינת ישראל             2015   
3  רע"פ 8866/15       רע"פ 8866/15 חאזמ רדיע נ. מדינת ישראל             2015   
4  רע"פ 1094/15    רע"פ 1094/15 שרית יחיאלוב נ. מדינת ישראל             2015   

                                 

In [7]:
combined_df

,level_0,index,גוף המסמך,מספר הליך,שם הליך,שנת פתיחת ההליך,גוף המסמך חתוך,predicted_probability,prediction_original_model,בקשה לרשות ערעור התקבלה?,...,טקסט שמצביע על תוצאת בקשה שונה (טקסטים שונים מופרדים ב-**),הערות תוצאת ערעור,prediction_adjusted_threshold\nבקשת רשות ערעור התקבלה?,גוף המסמך חתוך מסווג שני,prediction_second_model,second_model_predicted_probability,prediction_second_model_adjusted\nערעור התקבל?,prediction_adjusted_threshold\nבקשת רשות ערעור,second_model_predicted_probability\nתוצאת הערעור,Unnamed: 13
0,0,2977,"['החלטה בתיק רע""פ 5389/15 בבית המשפט העליון רע...","רע""פ 5389/15","רע""פ 5389/15",2015,"ישראל (24.3.1997); בר""ע 2853/91 פלוני נ\' מדינ...",0.016466,0,נדחה,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2978,"['החלטה בתיק רע""פ 2487/15 בבית המשפט העליון רע...","רע""פ 2487/15","רע""פ 2487/15 אייל אבולפיה נ. עיריית רמת גנ",2015,ופה המוקנית לתובע לצורך מתן מענה לבקשה לביטול ...,0.046868,0,נדחה,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2979,"['החלטה בתיק רע""פ 4086/15 בבית המשפט העליון רע...","רע""פ 4086/15","רע""פ 4086/15 עבד אגבאריה נ. מדינת ישראל",2015,"וני להכשרת המבנה, הרי שהאינטרס הציבורי של דחיק...",0.029892,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,2980,"['פסק דין בתיק רע""פ 8866/15 בבית המשפט העליון ...","רע""פ 8866/15","רע""פ 8866/15 חאזמ רדיע נ. מדינת ישראל",2015,"בין רכיב המאסר לרכיב הפסילה, יש מקום לשוב ולד...",0.958560,1,התקבל,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,2981,"['החלטה בתיק רע""פ 1094/15 בבית המשפט העליון רע...","רע""פ 1094/15","רע""פ 1094/15 שרית יחיאלוב נ. מדינת ישראל",2015,. עוד אציע לחברותיי כי עונש המאסר בפועל שהושת ...,0.365762,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8126,2322,6738,"['החלטה בתיק רע""פ 8581/11 בבית המשפט העליון רע...","רע""פ 8581/11","רע""פ 8581/11 יוספ ניסימ סטיונ נ. מדינת ישראל",2011,"ב: מדינת ישראל בשם המבקש: עו""ד יואב ציוני החלט...",0.288392,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
8127,2323,6739,"['החלטה בתיק רע""פ 1728/11 בבית המשפט העליון רע...","רע""פ 1728/11","רע""פ 1728/11 מאיר לוי נ. מדינת ישראל",2011,ידת מהימנות העדים שהעידו בפני הערכאה הדיונית (...,0.055941,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
8128,2324,6740,"['החלטה בתיק רע""פ 3371/11 בבית המשפט העליון רע...","רע""פ 3371/11","רע""פ 3371/11 אור וקנינ נ. התובע הצבאי הראשי",2011,צדיק מתן רשות ערעור לבית משפט זה בשים לב לכך ש...,0.024084,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
8129,2325,6741,"['החלטה בתיק רע""פ 3352/11 בבית המשפט העליון רע...","רע""פ 3352/11","רע""פ 3352/11 אריה כדורי נ. מדינת ישראל",2011,"1174/97 רפאלי נ\' מדינת ישראל (לא פורסם, 24.3...",0.017643,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN


In [9]:
data = combined_df.dropna(subset = ['בקשה לרשות ערעור התקבלה?'])

In [10]:
data

,level_0,index,גוף המסמך,מספר הליך,שם הליך,שנת פתיחת ההליך,גוף המסמך חתוך,predicted_probability,prediction_original_model,בקשה לרשות ערעור התקבלה?,...,טקסט שמצביע על תוצאת בקשה שונה (טקסטים שונים מופרדים ב-**),הערות תוצאת ערעור,prediction_adjusted_threshold\nבקשת רשות ערעור התקבלה?,גוף המסמך חתוך מסווג שני,prediction_second_model,second_model_predicted_probability,prediction_second_model_adjusted\nערעור התקבל?,prediction_adjusted_threshold\nבקשת רשות ערעור,second_model_predicted_probability\nתוצאת הערעור,Unnamed: 13
0,0,2977,"['החלטה בתיק רע""פ 5389/15 בבית המשפט העליון רע...","רע""פ 5389/15","רע""פ 5389/15",2015,"ישראל (24.3.1997); בר""ע 2853/91 פלוני נ\' מדינ...",0.016466,0,נדחה,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2978,"['החלטה בתיק רע""פ 2487/15 בבית המשפט העליון רע...","רע""פ 2487/15","רע""פ 2487/15 אייל אבולפיה נ. עיריית רמת גנ",2015,ופה המוקנית לתובע לצורך מתן מענה לבקשה לביטול ...,0.046868,0,נדחה,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,2980,"['פסק דין בתיק רע""פ 8866/15 בבית המשפט העליון ...","רע""פ 8866/15","רע""פ 8866/15 חאזמ רדיע נ. מדינת ישראל",2015,"בין רכיב המאסר לרכיב הפסילה, יש מקום לשוב ולד...",0.958560,1,התקבל,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,2982,"['החלטה בתיק רע""פ 3622/15 בבית המשפט העליון רע...","רע""פ 3622/15","רע""פ 3622/15 יעקב סארה נ. מדינת ישראל",2015,נש שעליו הוריתי – מבוטל איפוא. המבקש יתייצב לר...,0.541677,1,נדחה,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6,2983,"['החלטה בתיק רע""פ 1516/15 בבית המשפט העליון רע...","רע""פ 1516/15","רע""פ 1516/15 נקישוב דימטרי נ. מדינת ישראל",2015,"הוא לא הבין את תוכנו ומהותו של התצהיר, עליו ח...",0.096509,0,נדחה,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5455,2984,4266,"['החלטה בתיק רע""פ 133/13 בבית המשפט העליון רע""...","רע""פ 133/13","רע""פ 133/13 ציקו עבדי נ. מדינת ישראל",2013,"הבלתי חוקיים באורך רוח ראוי לציון, ואולי הבעי...",0.201271,0,נדחה,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5474,3006,4288,"['החלטה בתיק רע""פ 182/13 בבית המשפט העליון רע""...","רע""פ 182/13","רע""פ 182/13 נעמ משה נ. מדינת ישראל",2013,"נית בגזר דינה, כהוא זה, מרף הענישה הראוי בנסיב...",0.047010,0,נדחה,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5592,3140,7361,"['החלטה בתיק רע""פ 1370/17 בבית המשפט העליון רע...","רע""פ 1370/17","רע""פ 1370/17",2017,"סר. 9. אשר על כן, בקשת רשות הערעור נדחית בזאת....",0.028190,0,נדחה,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5606,3154,7375,"['החלטה בתיק רע""פ 3183/17 בבית המשפט העליון רע...","רע""פ 3183/17","רע""פ 3183/17 יואל קרויז נ. מדינת ישראל",2017,"851/13 עודה נ\' מדינת ישראל (25.10.2015); ע""פ ...",0.112401,0,נדחה,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
data['רע"פ בקשה אחרת או דלמטה מיוחד'].value_counts()


,count
"רע""פ בקשה אחרת או דלמטה מיוחד",
בקשה אחרת,83
דלמטה מיוחד,15


In [13]:
data[(data['רע"פ בקשה אחרת או דלמטה מיוחד'] == 'בקשה אחרת') | (data['רע"פ בקשה אחרת או דלמטה מיוחד'] == 'דלמטה מיוחד')].shape[0]

98

In [19]:
data["אחר"] = data.apply(
    lambda row: 1 if row["בקשה לרשות ערעור התקבלה?"] == "אחר" or not pd.isna(row['רע"פ בקשה אחרת או דלמטה מיוחד']) else 0, axis=1
)

<ipython-input-19-188207f6ee3b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["אחר"] = data.apply(


In [20]:
data["אחר"].value_counts()

,count
אחר,
0,567
1,135


# Embedding

In [22]:
tokenizer = AutoTokenizer.from_pretrained("dean-ai/sentence_transformer_Legal-heBERT")
model = AutoModel.from_pretrained("dean-ai/sentence_transformer_Legal-heBERT")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.86M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/678 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/498M [00:00<?, ?B/s]

In [23]:
def get_embeddings(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1).detach().numpy()
    return embeddings[0]

In [25]:

# print("מתחילים לחשב אימבדינגים על גוף המסמך המלא...")
# data["גוף המסמך המלא - embeddings"] = data["גוף המסמך"].apply(lambda text: get_embeddings(text))
# print("אימבדינגים 1 חושבו בהצלחה!")


print("מתחילים לחשב אימבדינגים על גוף המסמך החתוך...")
data["גוף המסמך חתוך - embeddings"] = data["גוף המסמך חתוך"].apply(lambda text: get_embeddings(text))
print("אימבדינגים 2 חושבו בהצלחה!")


מתחילים לחשב אימבדינגים על גוף המסמך החתוך...
אימבדינגים 2 חושבו בהצלחה!


<ipython-input-25-0a44db08ea2d>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["גוף המסמך חתוך - embeddings"] = data["גוף המסמך חתוך"].apply(lambda text: get_embeddings(text))


In [26]:
output_file = "/content/drive/My Drive/all_OTHER_cases_with_embeddings.xlsx"
data.to_excel(output_file, index=False)
print(f"Combined file saved at: {output_file}")

Combined file saved at: /content/drive/My Drive/all_OTHER_cases_with_embeddings.xlsx


# Split and train classifier

In [36]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report

# הכנת הנתונים
X = np.array(data["גוף המסמך חתוך - embeddings"].tolist())  # הפיכת העמודה לרשימה
y = data["אחר"].values

# הגדרת משקלות לסיווג לא מאוזן
class_weights = {0: 1, 1: 2}

# יצירת המודל של Logistic Regression
classifier = LogisticRegression(class_weight=class_weights, random_state=42, max_iter=1000)
classifier.fit(X, y)

# חיזוי על אותם נתונים
y_pred = classifier.predict(X)

# חיזוי הסתברויות
y_pred_prob = classifier.predict_proba(X)[:, 1]

# חישוב מטריצת בלבול
conf_matrix = confusion_matrix(y, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

# דוח ביצועים
report = classification_report(y, y_pred)
print("\nClassification Report:")
print(report)

tn, fp, fn, tp = conf_matrix.ravel()

# הדפסת הקונפיושן מטריקס עם תוויות
print("Confusion Matrix with Labels:")
print(f"True Negatives (TN): {tn}")
print(f"False Positives (FP): {fp}")
print(f"False Negatives (FN): {fn}")
print(f"True Positives (TP): {tp}")

print("\nConfusion Matrix:")
print(f"[[{tn} {fp}]")
print(f" [{fn} {tp}]]")


# חישוב מדדים נוספים אם נדרש
accuracy = (tn + tp) / (tn + fp + fn + tp)
print(f"\nAccuracy: {accuracy:.2f}")


joblib.dump(classifier, 'other_cases_classifer.pkl')

Confusion Matrix:
[[555  12]
 [ 19 116]]

Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.98      0.97       567
           1       0.91      0.86      0.88       135

    accuracy                           0.96       702
   macro avg       0.94      0.92      0.93       702
weighted avg       0.96      0.96      0.96       702

Confusion Matrix with Labels:
True Negatives (TN): 555
False Positives (FP): 12
False Negatives (FN): 19
True Positives (TP): 116

Confusion Matrix:
[[555 12]
 [19 116]]

Accuracy: 0.96


In [38]:
def export_model_errors(data, filename="model_errors.xlsx"):
    """
    Exports FP and FN errors to an Excel file with the document body and prediction probabilities.

    Parameters:
    - data (DataFrame): The DataFrame containing model predictions and probabilities.
    - filename (str): The name of the output Excel file.

    Returns:
    - None
    """
    # טבלת False Negatives: הערך בפועל 1 אך סווג כ-0
    fn_table = data[(data["אחר"] == 1) & (data["Prediction"] == 0)][["גוף המסמך", "Prediction_Prob"]]
    fn_table["Error Type"] = "FN"  # סוג הטעות

    # טבלת False Positives: הערך בפועל 0 אך סווג כ-1
    fp_table = data[(data["אחר"] == 0) & (data["Prediction"] == 1)][["גוף המסמך", "Prediction_Prob"]]
    fp_table["Error Type"] = "FP"  # סוג הטעות

    # איחוד הטבלאות
    errors_table = pd.concat([fn_table, fp_table], ignore_index=True)


    output_file = "/content/drive/My Drive/Other_cases_classifiers_errors_only_TP_TN.xlsx"  # עדכן את הנתיב לשמירת הקובץ
    errors_table.to_excel(output_file, index=False)
    print(f"Combined file saved at: {output_file}")

export_model_errors(data, filename="model_errors.xlsx")


Combined file saved at: /content/drive/My Drive/Other_cases_classifiers_errors_only_TP_TN.xlsx


In [39]:
joblib.dump(classifier, 'other_cases_classifer.pkl')

['other_cases_classifer.pkl']